In [1]:
from fastai.text import *
import pickle
from scipy import ndimage
from numpy import linalg as LA
import scipy.spatial as sp
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import sys
sys.path.insert(1, '/home/gaurav/Downloads/ISL/img2vec')
from img_to_vec import Img2Vec
from PIL import Image

In [2]:
img2vec = Img2Vec(cuda=True)

/home/gaurav/anaconda3/envs/fastai-bleed/lib/python3.7/site-packages/torchvision/transforms/transforms.py:208: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [3]:
def find_cos_similarity(one, two):
    pic_one = Image.open(one)
    pic_two = Image.open(two)
    pic_one_vec = img2vec.get_vec(pic_one)
    pic_two_vec = img2vec.get_vec(pic_two)
    cos_sim = cosine_similarity(pic_one_vec.reshape(1,-1), pic_two_vec.reshape(1,-1))
    return cos_sim[0][0]

In [5]:
root_dir = Path('/home/gaurav/Downloads/ISL/open_pose_frames')
all_classes = []
for cls in root_dir.ls():
    all_classes.append(str(cls).split('/')[-1])
print(f'Total No. of classes: {len(all_classes)}')
print(all_classes)

Total No. of classes: 29
['letter', 'train', 'transportation', 'mouse', 'thank_you', 'season', 'it', 'old', 'bill', 'bus', 'fast', 'soap', 'ring', 'you', 'boat', 'television', 'they', 'dry', 'week', 'hello', 'laptop', 'clock', 'truck', 'spring', 'how_are_you', 'shirt', 'happy', 'science', 'quiet']


In [8]:
# searching in all classes
correct_preds = 0
for sub_cls_path in root_dir.ls():
    # searching in all videos of that class
    for sub_vid_path in sub_cls_path.ls():
        total_frames = len(sub_vid_path.ls())
        pred_score = {}
        count_score = {}
        for cls in all_classes:
            pred_score[cls] = 0
            count_score[cls] = 0
        # searching in all frames of that video
        for img_path in sub_vid_path.ls():
            if total_frames//2 - 5 <= int(re.search(r'image(\d+)', str(img_path)).group(1)) <= total_frames//2 + 5:
                # because this frame occurs in middle, it might be relevant, so lets try matching it
                # trying to match the relevant image with all classes          
                for cls_path in root_dir.ls():
                    for vid_path in cls_path.ls():
                        if vid_path == sub_vid_path:
                            continue
                        total_target_vid_frames = len(vid_path.ls())
                        for vid_frame_path in vid_path.ls():
                            if total_target_vid_frames//2 - 5 <= int(re.search(r'image(\d+)', str(vid_frame_path)).group(1)) <= total_target_vid_frames//2 + 5:
                                if find_cos_similarity(vid_frame_path, img_path) >= 0.95:
                                    pred_score[str(vid_frame_path).split('/')[-3]] += 1
                        count_score[str(vid_frame_path).split('/')[-3]] += 1 
        actual_label = str(img_path).split("/")[-3]
        print(f'Actual: {actual_label}')
        for key in pred_score.keys():
            if count_score[key]:
                pred_score[key] = pred_score[key]/(count_score[key]*10)
        print(f'Predicted: {pred_score}')
        if actual_label == max(pred_score, key=pred_score.get):
            correct_preds += 1

Actual: letter
Predicted: {'letter': 0.43636363636363634, 'train': 0.05909090909090909, 'transportation': 0.25, 'mouse': 0.08484848484848485, 'thank_you': 0.37727272727272726, 'season': 0.12272727272727273, 'it': 0.09545454545454546, 'old': 0.013636363636363636, 'bill': 0.4090909090909091, 'bus': 0.19090909090909092, 'fast': 0.09318181818181819, 'soap': 0.2318181818181818, 'ring': 0.2606060606060606, 'you': 0.18484848484848485, 'boat': 0.29545454545454547, 'television': 0.45227272727272727, 'they': 0.013636363636363636, 'dry': 0.22272727272727272, 'week': 0.39545454545454545, 'hello': 0.12424242424242424, 'laptop': 0.22121212121212122, 'clock': 0.43636363636363634, 'truck': 0.08484848484848485, 'spring': 0.39090909090909093, 'how_are_you': 0.296969696969697, 'shirt': 0.42272727272727273, 'happy': 0.23636363636363636, 'science': 0.2590909090909091, 'quiet': 0.2863636363636364}
Actual: letter
Predicted: {'letter': 0.36666666666666664, 'train': 0.15, 'transportation': 0.01818181818181818,

Actual: mouse
Predicted: {'letter': 0.03636363636363636, 'train': 0.5227272727272727, 'transportation': 0.0, 'mouse': 0.759090909090909, 'thank_you': 0.0, 'season': 0.05, 'it': 0.25, 'old': 0.04090909090909091, 'bill': 0.24545454545454545, 'bus': 0.0, 'fast': 0.3, 'soap': 0.0, 'ring': 0.05757575757575758, 'you': 0.1696969696969697, 'boat': 0.13636363636363635, 'television': 0.06818181818181818, 'they': 0.045454545454545456, 'dry': 0.0, 'week': 0.06363636363636363, 'hello': 0.048484848484848485, 'laptop': 0.0, 'clock': 0.01818181818181818, 'truck': 0.024242424242424242, 'spring': 0.07272727272727272, 'how_are_you': 0.021212121212121213, 'shirt': 0.0, 'happy': 0.05, 'science': 0.16363636363636364, 'quiet': 0.0}
Actual: thank_you
Predicted: {'letter': 0.2909090909090909, 'train': 0.05909090909090909, 'transportation': 0.13636363636363635, 'mouse': 0.0, 'thank_you': 0.39090909090909093, 'season': 0.31363636363636366, 'it': 0.25, 'old': 0.2681818181818182, 'bill': 0.2, 'bus': 0.195454545454

Actual: bus
Predicted: {'letter': 0.08636363636363636, 'train': 0.013636363636363636, 'transportation': 0.00909090909090909, 'mouse': 0.0030303030303030303, 'thank_you': 0.2409090909090909, 'season': 0.05909090909090909, 'it': 0.0, 'old': 0.00909090909090909, 'bill': 0.004545454545454545, 'bus': 0.7909090909090909, 'fast': 0.004545454545454545, 'soap': 0.0, 'ring': 0.0, 'you': 0.07878787878787878, 'boat': 0.031818181818181815, 'television': 0.11363636363636363, 'they': 0.0, 'dry': 0.00909090909090909, 'week': 0.1409090909090909, 'hello': 0.01818181818181818, 'laptop': 0.045454545454545456, 'clock': 0.022727272727272728, 'truck': 0.00909090909090909, 'spring': 0.22272727272727272, 'how_are_you': 0.06363636363636363, 'shirt': 0.03636363636363636, 'happy': 0.11363636363636363, 'science': 0.03636363636363636, 'quiet': 0.0}
Actual: bus
Predicted: {'letter': 0.42272727272727273, 'train': 0.21363636363636362, 'transportation': 0.06363636363636363, 'mouse': 0.10303030303030303, 'thank_you': 0.

Actual: ring
Predicted: {'letter': 0.07954545454545454, 'train': 0.013636363636363636, 'transportation': 0.07272727272727272, 'mouse': 0.12727272727272726, 'thank_you': 0.17727272727272728, 'season': 0.16363636363636364, 'it': 0.004545454545454545, 'old': 0.03636363636363636, 'bill': 0.41818181818181815, 'bus': 0.1409090909090909, 'fast': 0.08181818181818182, 'soap': 0.045454545454545456, 'ring': 0.41818181818181815, 'you': 0.0696969696969697, 'boat': 0.08636363636363636, 'television': 0.2159090909090909, 'they': 0.0, 'dry': 0.16363636363636364, 'week': 0.19545454545454546, 'hello': 0.1484848484848485, 'laptop': 0.16666666666666666, 'clock': 0.05909090909090909, 'truck': 0.030303030303030304, 'spring': 0.01818181818181818, 'how_are_you': 0.07575757575757576, 'shirt': 0.20454545454545456, 'happy': 0.11818181818181818, 'science': 0.19772727272727272, 'quiet': 0.004545454545454545}
Actual: you
Predicted: {'letter': 0.056818181818181816, 'train': 0.05909090909090909, 'transportation': 0.0,

Actual: they
Predicted: {'letter': 0.02727272727272727, 'train': 0.07272727272727272, 'transportation': 0.004545454545454545, 'mouse': 0.13333333333333333, 'thank_you': 0.01818181818181818, 'season': 0.06363636363636363, 'it': 0.07727272727272727, 'old': 0.20454545454545456, 'bill': 0.022727272727272728, 'bus': 0.004545454545454545, 'fast': 0.14545454545454545, 'soap': 0.013636363636363636, 'ring': 0.021212121212121213, 'you': 0.07575757575757576, 'boat': 0.00909090909090909, 'television': 0.05909090909090909, 'they': 0.24545454545454545, 'dry': 0.045454545454545456, 'week': 0.013636363636363636, 'hello': 0.15757575757575756, 'laptop': 0.03636363636363636, 'clock': 0.08636363636363636, 'truck': 0.0, 'spring': 0.013636363636363636, 'how_are_you': 0.048484848484848485, 'shirt': 0.013636363636363636, 'happy': 0.04090909090909091, 'science': 0.06136363636363636, 'quiet': 0.13636363636363635}
Actual: they
Predicted: {'letter': 0.004545454545454545, 'train': 0.0, 'transportation': 0.0, 'mous

Actual: laptop
Predicted: {'letter': 0.33636363636363636, 'train': 0.1409090909090909, 'transportation': 0.11818181818181818, 'mouse': 0.012121212121212121, 'thank_you': 0.2545454545454545, 'season': 0.11818181818181818, 'it': 0.00909090909090909, 'old': 0.1590909090909091, 'bill': 0.09545454545454546, 'bus': 0.20909090909090908, 'fast': 0.10681818181818181, 'soap': 0.3090909090909091, 'ring': 0.2, 'you': 0.15757575757575756, 'boat': 0.022727272727272728, 'television': 0.6318181818181818, 'they': 0.04090909090909091, 'dry': 0.23636363636363636, 'week': 0.5045454545454545, 'hello': 0.14545454545454545, 'laptop': 0.39090909090909093, 'clock': 0.13636363636363635, 'truck': 0.04242424242424243, 'spring': 0.2863636363636364, 'how_are_you': 0.2818181818181818, 'shirt': 0.30454545454545456, 'happy': 0.1590909090909091, 'science': 0.4113636363636364, 'quiet': 0.25}
Actual: laptop
Predicted: {'letter': 0.20909090909090908, 'train': 0.013636363636363636, 'transportation': 0.20454545454545456, 'm

Actual: how_are_you
Predicted: {'letter': 0.2340909090909091, 'train': 0.17272727272727273, 'transportation': 0.16363636363636364, 'mouse': 0.051515151515151514, 'thank_you': 0.30454545454545456, 'season': 0.11363636363636363, 'it': 0.0, 'old': 0.00909090909090909, 'bill': 0.18181818181818182, 'bus': 0.16363636363636364, 'fast': 0.06363636363636363, 'soap': 0.11818181818181818, 'ring': 0.16363636363636364, 'you': 0.08181818181818182, 'boat': 0.25, 'television': 0.325, 'they': 0.013636363636363636, 'dry': 0.1, 'week': 0.33636363636363636, 'hello': 0.04242424242424243, 'laptop': 0.1606060606060606, 'clock': 0.19545454545454546, 'truck': 0.21212121212121213, 'spring': 0.3181818181818182, 'how_are_you': 0.4, 'shirt': 0.4, 'happy': 0.2818181818181818, 'science': 0.18636363636363637, 'quiet': 0.2727272727272727}
Actual: how_are_you
Predicted: {'letter': 0.22272727272727272, 'train': 0.30454545454545456, 'transportation': 0.1318181818181818, 'mouse': 0.10909090909090909, 'thank_you': 0.363636

Actual: quiet
Predicted: {'letter': 0.32727272727272727, 'train': 0.6636363636363637, 'transportation': 0.1, 'mouse': 0.11818181818181818, 'thank_you': 0.1318181818181818, 'season': 0.3409090909090909, 'it': 0.0, 'old': 0.17727272727272728, 'bill': 0.2409090909090909, 'bus': 0.045454545454545456, 'fast': 0.3704545454545455, 'soap': 0.20909090909090908, 'ring': 0.09696969696969697, 'you': 0.17575757575757575, 'boat': 0.4590909090909091, 'television': 0.475, 'they': 0.08636363636363636, 'dry': 0.5181818181818182, 'week': 0.4590909090909091, 'hello': 0.1696969696969697, 'laptop': 0.2696969696969697, 'clock': 0.2409090909090909, 'truck': 0.3787878787878788, 'spring': 0.16818181818181818, 'how_are_you': 0.403030303030303, 'shirt': 0.2590909090909091, 'happy': 0.6090909090909091, 'science': 0.3477272727272727, 'quiet': 0.8909090909090909}
Actual: quiet
Predicted: {'letter': 0.3068181818181818, 'train': 0.01818181818181818, 'transportation': 0.0, 'mouse': 0.03939393939393939, 'thank_you': 0.2

In [13]:
total_vids = 0
for sub_cls_path in root_dir.ls():
    total_vids += len(sub_cls_path.ls())
print(total_vids)

73


In [14]:
print(f'Accuracy: {correct_preds/total_vids}')

Accuracy: 0.5068493150684932
